In [1]:
import pandas as pd


In [128]:
play_csv=pd.read_csv("./play_2005.csv")

# Throw out all plays that occur in second half...
play1st_csv=play_csv[play_csv["Period Number"]<3]
print(len(play1st_csv))
play1st_csv.head(10)

68622


,Game Code,Play Number,Period Number,Clock,Offense Team Code,Defense Team Code,Offense Points,Defense Points,Down,Distance,Spot,Play Type,Drive Number,Drive Play
0,690002820050901,1,1,900.0,690,28,0,0,NaN,NaN,65,KICKOFF,NaN,NaN
1,690002820050901,2,1,NaN,28,690,0,0,1.0,10.0,80,RUSH,1.0,1.0
2,690002820050901,3,1,NaN,28,690,0,0,2.0,8.0,78,PASS,1.0,1.0
3,690002820050901,4,1,NaN,28,690,0,0,3.0,8.0,78,PENALTY,1.0,0.0
4,690002820050901,5,1,NaN,28,690,0,0,3.0,19.0,89,PASS,1.0,1.0
5,690002820050901,6,1,NaN,28,690,0,0,4.0,19.0,89,PUNT,NaN,NaN
6,690002820050901,7,1,826.0,690,28,0,0,1.0,10.0,54,PASS,2.0,1.0
7,690002820050901,8,1,NaN,690,28,0,0,2.0,5.0,49,RUSH,2.0,1.0
8,690002820050901,9,1,NaN,690,28,0,0,3.0,5.0,49,PENALTY,2.0,1.0
9,690002820050901,10,1,NaN,690,28,0,0,1.0,10.0,44,RUSH,2.0,1.0


In [129]:
# List of games in the csv - can use later for iteration
play1st_csv['Game Code'].unique()

array([ 690002820050901,  204014020050901,   86016420050901,
        529028820050901,  786038820050901,  508049820050901,
        128064820050901,  771070920050901,  428071920050901,
        736074920050901,  295075420050901,  306012920050902,
         29073220050902,  419000820050903,  434003020050903,
        669003120050903,  255003720050903,   67007720050903,
        102010720050903,  697014720050903,  156015720050903,
        193019620050903,  811023520050903,   66025720050903,
        657027720050903,  587030120050903,  299031120050903,
         47031220050903,  231032720050903,  229032820050903,
        331041620050903,  503041820050903,  454043020050903,
        380046320050903,  704047220050903,  519050920050903,
        414051820050903,  440052120050903,  698052220050903,
        550052820050903,  651053920050903,  513054520050903,
        110062620050903,  207063020050903,   51066320050903,
          9069420050903,  671070320050903,   90071620050903,
        756072120050903,

In [237]:
# Write a function that looks through the file and returns the offensive yards for each team in the game half
# Returns a dictionary with rush and pass stats for each team (right now)
def half_stats(gameNum):
    half_csv=play1st_csv[play1st_csv["Game Code"]==gameNum].reset_index()
    team1,team2 = half_csv["Offense Team Code"].unique()
    
    #create dictionary for stats
    stats={
        team1:{'RUSH':0,
               'RUSH YARDS':0,
               'PASS':0,
               'INC PASS':0,
               'PASS YARDS':0,
               'PENALTY':0,
               'PENALTY YARDS':0,
               'POINTS':0
        },
        team2:{'RUSH':0,
               'RUSH YARDS':0,
               'PASS':0,
               'INC PASS':0,
               'PASS YARDS':0,
               'PENALTY':0,
               'PENALTY YARDS':0,
               'POINTS':0
        }
    }
    
    numPlays=len(half_csv)
    for index, row in half_csv.iterrows():
        if(index+1<numPlays):
            team=half_csv.iloc[index]['Offense Team Code']
            playType=half_csv.iloc[index]['Play Type']
            spot1=half_csv.iloc[index]['Spot']
            spot2=half_csv.iloc[index+1]['Spot']
            numYards=spot1-spot2
            # Update Dictionary
            if(playType=='RUSH'):
                stats[team][playType]=stats[team][playType]+1
                stats[team]['RUSH YARDS']=stats[team]['RUSH YARDS']+numYards
            if(playType=='PASS'):
                stats[team][playType]=stats[team][playType]+1
                stats[team]['PASS YARDS']=stats[team]['PASS YARDS']+numYards
                if(numYards==0):
                    stats[team]['INC PASS']=stats[team]['INC PASS']+1
            if(playType=='PENALTY'):
                if(numYards>0):
                    stats[team][playType]=stats[team][playType]+1
                    stats[team]['PENALTY YARDS']=stats[team]['PENALTY YARDS']+numYards
                else:
                    team=half_csv.iloc[index]['Defense Team Code']
                    stats[team][playType]=stats[team][playType]+1
                    stats[team]['PENALTY YARDS']=stats[team]['PENALTY YARDS']-numYards

    # Get the halftime scores for each team
    team=half_csv.tail(1)['Offense Team Code'].values[0]
    teamPoints=half_csv.tail(1)['Offense Points'].values[0]
    stats[team]['POINTS']=half_csv.tail(1)['Offense Points'].values[0]
    team=half_csv.tail(1)['Defense Team Code'].values[0]
    teamPoints=half_csv.tail(1)['Defense Points'].values[0]
    stats[team]['POINTS']=half_csv.tail(1)['Defense Points'].values[0]

    return (stats)


In [238]:
result=half_stats(690002820050901)
result_df=pd.DataFrame(data=result)
result_df


,690,28
RUSH,20,19
RUSH YARDS,34,136
PASS,15,19
INC PASS,7,8
PASS YARDS,72,48
PENALTY,5,0
PENALTY YARDS,36,0
POINTS,2,28


In [239]:
result=half_stats(231032720050903)
result_df=pd.DataFrame(data=result)
result_df

,327,231
RUSH,18,15
RUSH YARDS,78,28
PASS,17,15
INC PASS,7,8
PASS YARDS,65,64
PENALTY,5,5
PENALTY YARDS,30,35
POINTS,13,7
